In [ ]:
import numpy as np
import tifffile as tiff
from pathlib import Path
import flammkuchen as fl
import os.path
import matplotlib.pyplot as plt
import json

In [ ]:
master =  Path(r"\\Funes\Shared\experiments\E0071_lotr\full_ring")
files_path = list(master.glob("*_f*/*f*/anatomy_suite2p_res.nrrd"))
num_fish = len(files_path)
print(num_fish)

In [ ]:
def get_xform_values(xform_path):
    with open(xform_path) as f:
        lines = f.readlines()
    lines = lines[6:11]
    
    trans_values = np.zeros((5, 3))
    for i in range(5):
        tmp_line = lines[i]
        tmp_line = tmp_line[8:-2]
        tmp_line = tmp_line.split()
        tmp_line = [float(j) for j in tmp_line]
        trans_values[i] = tmp_line
        
    return trans_values

In [ ]:
def values2matrix(trasformation_vals):
        
    # translation:
    m_trans = np.identity(4)
    for i in range(3):
        m_trans[i,3] = trasformation_vals[0,i]
    
    # rotation:
    # in cmtk the rotation values are ordered: x, y, z
    m_rot_z = np.identity(4)
    theta = np.deg2rad(trasformation_vals[1, 2])
    m_rot_z[0,0] = np.cos(theta)
    m_rot_z[0,1] = -1 * np.sin(theta)
    m_rot_z[1,0] = np.sin(theta)
    m_rot_z[1,1] = np.cos(theta)
    
    m_rot_x = np.identity(4)
    theta = np.deg2rad(trasformation_vals[1, 0])
    m_rot_x[1,1] = np.cos(theta)
    m_rot_x[1,2] = -1 * np.sin(theta)
    m_rot_x[2,1] = np.sin(theta)
    m_rot_x[2,2] = np.cos(theta)
    
    m_rot_y = np.identity(4)
    theta = np.deg2rad(trasformation_vals[1, 1])
    m_rot_y[0,0] = np.cos(theta)
    m_rot_y[2,0] = -1 * np.sin(theta)
    m_rot_y[0,2] = np.sin(theta)
    m_rot_y[2,2] = np.cos(theta)
    
    m_rot = np.identity(4)
    alpha = np.deg2rad(trasformation_vals[1, 0])
    theta = np.deg2rad(trasformation_vals[1, 1])
    phi = np.deg2rad(trasformation_vals[1, 2])
    cos0 = np.cos(alpha)
    sin0 = np.sin(alpha)
    cos1 = np.cos(theta)
    sin1 = np.sin(theta)
    cos2 = np.cos(phi)
    sin2 = np.sin(phi)
    sin0xsin1 = sin0 * sin1
    cos0xsin1 = cos0 * sin1
    
    m_rot[0,0] = cos1 * cos2
    m_rot[1,0] = (-1 *cos1) * sin2
    m_rot[2,0] = (-1 * sin1) 
    m_rot[0,1] = sin0xsin1 * cos2 + cos0 * sin2
    m_rot[1,1] = (-1* sin0xsin1) * sin2 + cos0*cos2
    m_rot[2,1] = sin0 * cos1
    m_rot[0,2] = cos0xsin1 * cos2 - sin0*sin2
    m_rot[1,2] = -cos0xsin1 * sin2 - sin0 * cos2
    m_rot[2,2] = cos0 * cos1 
    
    # scale: 
    m_scale = np.identity(4)
    m_scale[0,0] = trasformation_vals[2,0]
    m_scale[1,1] = trasformation_vals[2,1]
    m_scale[2,2] = trasformation_vals[2,2]
    
    # shear: 
    # in cmtk the rotation values are ordered: x, y, z. I assume shear values are the same 
    m_shearx = np.identity(4)
    m_shearx[1,0] = trasformation_vals[3,0]
    m_sheary = np.identity(4)
    m_sheary[2,0] = trasformation_vals[3,1]
    m_shearz = np.identity(4)
    m_shearz[2,1] = trasformation_vals[3,2]
    
    m_shear = m_shearz @ m_sheary @ m_shearx

    return  m_shear @ m_scale @ m_rot @ m_trans 

In [ ]:
# For a single fish:
# Getting the coordinates: 
fish_path = Path(os.path.dirname(str(files_path[5])))
suite2p_file = fl.load(fish_path / "data_from_suite2p_unfiltered.h5")
coords_orig = suite2p_file['coords']
coords = np.ones((np.shape(coords_orig)[0], 4))
coords[:, 0:3] = coords_orig
stack_shape = np.shape(suite2p_file['anatomy_stack'])

# Getting the resolution values to scale the coordinates:
with open(next(Path(fish_path).glob("*metadata.json")), "r") as f:
        metadata = json.load(f)
lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
num_planes = lsconfig["triggering"]["n_planes"]
res = [z_tot_span / num_planes, 0.6, 0.6]
for i in range(3):
    coords[:, i] *= res[i]
    
# Getting the transformation values:
xform1_path = fish_path / "anatomy_affine_res_2avg.xform" / "registration"
trans_values1 = get_xform_values(xform1_path)
center_vals1 = trans_values1[4]

In [ ]:
# Transforming the coordinates: 
t_matrix1 = values2matrix(trans_values1)

num_coords = np.shape(coords)[0]
t_coords = np.ones_like(coords)
# Removing center values and reordering coordinates: 
for i in range(num_coords):
    t_coords[i,0] = coords[i,1] - center_vals1[1]
    t_coords[i,1] = coords[i,2] - center_vals1[2]
    t_coords[i,2] = coords[i,0] - center_vals1[0]
        
t_coords = t_matrix1 @ t_coords.T
t_coords = t_coords.T

tr_coords = np.ones_like(coords)
for i in range(num_coords):
    tr_coords[i,0] = t_coords[i,2] + center_vals1[2]
    tr_coords[i,1] = t_coords[i,0] + center_vals1[0]
    tr_coords[i,2] = t_coords[i,1] + center_vals1[1]

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(15,10))
num_cells = np.shape(coords)[1]
title_list = ['Horizontal', 'Lateral', 'Frontal']
for i in range(3):
    ax[0,i].set_title(title_list[i])
    for j in range(2):
        ax[j,i].axis('off')
        ax[j,i].set_ylim(-200, 1200)
        ax[j,i].set_xlim(-200, 1000)
ax[0,0].scatter(coords[:,1], coords[:,2], s=10)
ax[1,0].scatter(tr_coords[:,1], tr_coords[:,2], s=10)

ax[0,1].scatter(coords[:,2], coords[:,0], s=10)
ax[1,1].scatter(tr_coords[:,2], tr_coords[:,0], s=10)

ax[0,2].scatter(coords[:,1], coords[:,0], s=10)
ax[1,2].scatter(tr_coords[:,1], tr_coords[:,0], s=10)
plt.show()
fig.savefig(fish_path / "transformed_coords.jpg")

In [ ]:
# For all fish: 
for i in files_path:
    # Getting the coordinates: 
    fish_path = Path(os.path.dirname(str(i)))
    suite2p_file = fl.load(fish_path / "data_from_suite2p_unfiltered.h5")
    coords_orig = suite2p_file['coords']
    coords_orig = suite2p_file['coords']
    coords = np.ones((np.shape(coords_orig)[0], 4))
    coords[:, 0:3] = coords_orig
    
    # Getting the resolution values to scale the coordinates:
    with open(next(Path(fish_path).glob("*metadata.json")), "r") as f:
            metadata = json.load(f)
    lsconfig = metadata["imaging"]["microscope_config"]['lightsheet']['scanning']
    z_tot_span = lsconfig["z"]["piezo_max"] - lsconfig["z"]["piezo_min"]
    n_planes = lsconfig["triggering"]["n_planes"]
    res = [z_tot_span / n_planes, 0.6, 0.6]
    for i in range(3):
        coords[:, i] *= res[i]
    
    # Getting the transformation values:
    xform1_path = fish_path / "anatomy_affine_res_2avg.xform" / "registration"
    trans_values1 = get_xform_values(xform1_path)
    
    # Transforming the coordinates: 
    t_matrix1 = values2matrix(trans_values1)
    center_vals1 = trans_values1[4]
    num_coords = np.shape(coords)[0]
    t_coords = np.ones_like(coords)
    tr_coords = np.ones_like(coords)
    
    # Removing center values: 
    for i in range(num_coords):
        t_coords[i,0] = coords[i,1] - center_vals1[0]
        t_coords[i,1] = coords[i,2] - center_vals1[1]
        t_coords[i,2] = coords[i,0] - center_vals1[2]

    t_coords = t_matrix1 @ t_coords.T
    t_coords = t_coords.T

    for i in range(num_coords):
        tr_coords[i,0] = t_coords[i,2] + center_vals1[2]
        tr_coords[i,1] = t_coords[i,0] + center_vals1[0]
        tr_coords[i,2] = t_coords[i,1] + center_vals1[1]

    '''
    fig, ax = plt.subplots(2, 3, figsize=(15,10))
    num_cells = np.shape(coords)[1]
    title_list = ['Horizontal', 'Lateral', 'Frontal']
    for i in range(3):
        ax[0,i].set_title(title_list[i])
        for j in range(2):
            ax[j,i].axis('off')
            ax[j,i].set_ylim(-200, 1000)
            ax[j,i].set_xlim(-200, 1000)
    ax[0,0].scatter(coords[:,1], coords[:,2], s=10)
    ax[1,0].scatter(tr_coords[:,1], tr_coords[:,2], s=10)

    ax[0,1].scatter(coords[:,2], coords[:,0], s=10)
    ax[1,1].scatter(tr_coords[:,2], tr_coords[:,0], s=10)

    ax[0,2].scatter(coords[:,1], coords[:,0], s=10)
    ax[1,2].scatter(tr_coords[:,1], tr_coords[:,0], s=10)
    plt.show()
    fig.savefig(fish_path / "transformed_coords2.jpg")
    '''
    
    tr_coords = tr_coords[:,0:3]
    dir_t_coords = str(fish_path/'transformed_coords.h5')
    fl.save(dir_t_coords, {"transformed_coords": tr_coords})